In [2]:
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestRegressor
import numpy as np

files=['/content/drive/MyDrive/SEM6miniproject/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv','/content/drive/MyDrive/SEM6miniproject/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv']#,'/content/drive/MyDrive/SEM6miniproject/Friday-WorkingHours-Morning.pcap_ISCX.csv','/content/drive/MyDrive/SEM6miniproject/Monday-WorkingHours.pcap_ISCX.csv','/content/drive/MyDrive/SEM6miniproject/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv','/content/drive/MyDrive/SEM6miniproject/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv','/content/drive/MyDrive/SEM6miniproject/Tuesday-WorkingHours.pcap_ISCX.csv','/content/drive/MyDrive/SEM6miniproject/Wednesday-workingHours.pcap_ISCX.csv']
dfs=[]
for file in files:
    df_temp = pd.read_csv(file)
    dfs.append(df_temp)

df = pd.concat(dfs, ignore_index=True)


In [3]:
# Handling Missing Data (if any)
df = df.dropna()  # or use imputation based on the column's distribution

# Ensure there are no infinite values
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop or fill rows with NaN after replacing infinity
df = df.dropna()

# Encoding categorical columns (Label column)
label_encoder = LabelEncoder()
df[' Label'] = label_encoder.fit_transform(df[' Label'])

# Scaling numerical features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df.select_dtypes(include=['float64', 'int64']))

# Creating DataFrame from scaled features
df_scaled = pd.DataFrame(scaled_features, columns=df.select_dtypes(include=['float64', 'int64']).columns)

# Splitting dataset
X = df_scaled.drop(' Label', axis=1)
y = df_scaled[' Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:
y_train

,Label
417106,1.318906
328021,1.318906
98092,0.151004
63264,0.151004
51509,-1.016899
...,...
259178,-1.016899
365838,1.318906
131932,0.151004
146867,-1.016899


In [5]:
from sklearn.linear_model import LogisticRegression

# Train a simple logistic regression model
#model = LogisticRegression()
#model.fit(X_train, y_train)


In [6]:
# Initialize and train Random Forest Regressor
model = RandomForestRegressor(n_estimators=10, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(n_estimators=10, random_state=42)

In [7]:
# Make predictions
y_pred = model.predict(X_test)

In [11]:
from sklearn.metrics import mean_squared_error

# Assuming y_test and y_pred are defined
print("Random Forest Regression MSE:", mean_squared_error(y_test, y_pred))


Random Forest Regression MSE: 0.0002619738833016345


In [12]:
y_pred

array([ 0.15100377,  1.31890636,  0.15100377, ..., -1.01689883,
       -1.01689883,  1.31890636])

In [15]:
y_pred_rounded = np.round(y_pred).astype(int)

# Step 8: Clip the predicted values to ensure they are within the valid label range
y_pred_rounded = np.clip(y_pred_rounded, 0, len(label_encoder.classes_) - 1)

# Step 9: Inverse transform back to the original labels
y_pred_original = label_encoder.inverse_transform(y_pred_rounded)

# Step 10: Output the original predicted labels
print("Predicted Original Labels:", y_pred_original)

Predicted Original Labels: ['BENIGN' 'DDoS' 'BENIGN' ... 'BENIGN' 'BENIGN' 'DDoS']
